# One year of LISA data 

In [20]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os
from gap_study_utils.analysis_data import AnalysisData, get_suggested_tmax
from gap_study_utils.utils.signal_utils import waveform
from gap_study_utils.gaps.gap_funcs import generate_gap_ranges
from gap_study_utils.gaps import GapType
from gap_study_utils.mcmc_runner import run_mcmc
import matplotlib.pyplot as plt

np.random.seed(0)

LN_A = np.log(1e-21)
LN_F = np.log(0.005)
LN_FDOT = np.log(1e-9)

HOURS = 60 * 60
DAYS = 24 * HOURS

np.random.seed(0)
dt = 10
tmax = get_suggested_tmax(DAYS * 365.4)

outdir = f"outdir_1year"
os.makedirs(outdir, exist_ok=True)

gap_ranges = generate_gap_ranges(tmax, gap_period=DAYS * 14, gap_duration=HOURS * 7)
print("Number of gaps: ", len(gap_ranges))


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Number of gaps:  17


In [23]:
data = AnalysisData(
    data_kwargs=dict(dt=dt, noise=False, tmax=tmax),
    gap_kwargs=dict(type=GapType.STITCH, gap_ranges=gap_ranges),
    waveform_generator=waveform,
    waveform_parameters=[LN_A, LN_F, LN_FDOT],
);
print("DONEE")

[20:49:49] INFO     Initalizing GapWindow with STITCH gaps (17 gaps).                          ]8;id=578985;file:///Users/avaj0001/Documents/projects/pywavelet/case_studies.gaps/src/gap_study_utils/analysis_data.py\analysis_data.py]8;;\:]8;id=828180;file:///Users/avaj0001/Documents/projects/pywavelet/case_studies.gaps/src/gap_study_utils/analysis_data.py#129\129]8;;\

           INFO     AnalysisData initialized.                                                   ]8;id=494782;file:///Users/avaj0001/Documents/projects/pywavelet/case_studies.gaps/src/gap_study_utils/analysis_data.py\analysis_data.py]8;;\:]8;id=408897;file:///Users/avaj0001/Documents/projects/pywavelet/case_studies.gaps/src/gap_study_utils/analysis_data.py#81\81]8;;\

KeyboardInterrupt: 

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 3.3))
fig, _ = data.data_wavelet.plot(ax=ax, whiten_by=None, freq_range=[0.005, 0.028])
fig.savefig(os.path.join(outdir, "data_wavelet.png"), bbox_inches="tight")

![outdir_1year/data_wavelet.png](outdir_1year/data_wavelet.png)

In [22]:
%%timeit

data.lnl(LN_A, LN_F, LN_FDOT)

402 ms ± 12.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## MCMC

In [ ]:
run_mcmc(
    true_params=[LN_A, LN_F, LN_FDOT],
    gap_ranges=gap_ranges,
    gap_type="rectangular_window",
    Nf=64,
    tmax=tmax,
    dt=dt,
    alpha=0.0,
    highpass_fmin=None,
    frange=[0.005, 0.028],
    noise_realisation=False,
    outdir=f"{outdir}/mcmc_no_noise",
    noise_curve='TDI1',
    burnin=150,
    n_iter=250
)

![outdir_1year/mcmc_no_noise/summary.png](outdir_1year/mcmc_no_noise/summary.png)

In [ ]:
# Plotting both modes

lna = -48.395
lnf = (-4.0 * 1e-6)-5.29831
lnfdot = (-66.0 * 1e-6)-2.07232e1
mode2_wdm = data.htemplate(lna,lnf,lnfdot)
true_wdm = data.htemplate(LN_A,LN_F,LN_FDOT)

fig, axes = plt.subplots(3, 1, figsize=(12, 4), sharex=True, sharey=True)
true_wdm.plot(ax=axes[0], freq_range=[0.005, 0.028], label="True")
mode2_wdm.plot(ax=axes[1], freq_range=[0.005, 0.028], label="2ndary mode")
diffr = true_wdm - mode2_wdm
diffr.plot(ax=axes[2], freq_range=[0.005, 0.028], label="Difference")
plt.subplots_adjust(hspace=0.0)
plt.savefig(os.path.join(outdir, "true_vs_2ndary_mode.png"), bbox_inches="tight")

![outdir_1year/true_vs_2ndary_mode.png](outdir_1year/true_vs_2ndary_mode.png)